#### Import Statements

In [1]:
# import os
# import cv2
# import pandas as pd
# import numpy as np
# import imageio
# from imageio import v3 as Im
# from sklearn.metrics import confusion_matrix
# from keras.models import Sequential
# from keras.layers.core import Dense, Dropout, Activation, Flatten
# from keras.layers.convolutional import Convolution3D, MaxPooling3D
# from keras.optimizers import SGD, RMSprop
# from keras.callbacks import ModelCheckpoint
# from keras.utils import np_utils, generic_utils
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import confusion_matrix
# from sklearn import preprocessing
# from keras import backend as K
# import sys
# import tensorflow as tf

import os
import cv2
import pylab
import dlib
import numpy as np
import pandas as pd
#import imageio
import skimage.io as imageio
import sys
import tensorflow as tf
from tqdm import tqdm
from sklearn.metrics import confusion_matrix
from keras.models import Sequential, Model
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution3D, MaxPooling3D
#from keras.utils import multi_gpu_model
from keras.optimizers import SGD, RMSprop
from tensorflow.keras.optimizers import Adam
from keras.layers import Concatenate, Input, concatenate, add, multiply, maximum, LSTM, Reshape
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils, generic_utils
from sklearn.model_selection import train_test_split
#from sklearn import cross_validation
from sklearn import preprocessing
from keras import backend as K
import matplotlib.pyplot as plt
%matplotlib inline
#K.set_image_dim_ordering('th')
#K.set_image_data_format('channels_first')
#import matplotlib
#matplotlib.use('Agg')

#### GPU Check

In [2]:
gpu_available = tf.config.list_physical_devices('GPU')

In [3]:
gpu_available

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [5]:
#K.set_image_dim_ordering('th')
K.set_image_data_format('channels_first')

#### Initialization

In [6]:
predictor_path = "/Thesis/MicroExpSTCNN/shape_predictor_68_face_landmarks.dat"
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(predictor_path)

In [7]:
def get_landmark(img):
    rects = detector(img, 1)
    if len(rects) > 1:
        pass
    if len(rects) == 0:
        pass
    ans = np.matrix([[p.x, p.y] for p in predictor(img, rects[0]).parts()])
    return ans

In [8]:
def annotate_landmarks(img, landmarks, font_scale = 0.4):
    for idx, point in enumerate(landmarks):
        pos = (point[0, 0], point[0, 1])
        cv2.putText(img, str(idx), pos, fontFace=cv2.FONT_HERSHEY_SCRIPT_SIMPLEX, fontScale=font_scale, color=(0, 0, 255))
        cv2.circle(img, pos, 3, color=(0, 255, 255))
    return img

In [9]:
subjectPath = '/Thesis/Datasets/SMIC/HS'

In [38]:
image_rows, image_columns, frames_Count = 64, 64, 16
training_smic_list = []

#### SMIC Dataset

In [40]:
for lab in ['negative', 'positive', 'surprise']:
    subDirectory = os.listdir(subjectPath)
    count = 0
    for sub in tqdm(subDirectory):
        labelpath = '/Thesis/Datasets/SMIC/HS/'+ sub +'/micro/' + lab + '/'
        directorylisting = os.listdir(labelpath)
        for video in directorylisting:
            videopath = labelpath + video
            frames = []
            framelisting = os.listdir(videopath)
            if len(framelisting) > frames_Count:
                framerange = [x for x in range(frames_Count)]
                count = count + 1
                for frame in framerange:
                    imagepath = videopath + "/" + framelisting[frame]
                    image = cv2.imread(imagepath)
                    imageresize = cv2.resize(image, (image_rows, image_columns), interpolation = cv2.INTER_AREA)
                    grayimage = cv2.cvtColor(imageresize, cv2.COLOR_BGR2GRAY)
                    frames.append(grayimage)
                frames = np.asarray(frames)
                videoarray = np.rollaxis(np.rollaxis(frames, 2, 0), 2, 0)
                training_smic_list.append(videoarray)
                # break
    print(lab,count)
    # break
  

100%|██████████████████████████████████████████████████████████████████████████████████| 16/16 [00:00<00:00, 43.38it/s]


negative 69


100%|██████████████████████████████████████████████████████████████████████████████████| 16/16 [00:01<00:00, 10.74it/s]


positive 50


100%|██████████████████████████████████████████████████████████████████████████████████| 16/16 [00:03<00:00,  4.17it/s]

surprise 43


In [ ]:
len(training_smic_list)

In [ ]:
66+50

#### Creating Target labels

In [ ]:
training_smic_list = np.asarray(training_smic_list)
training_smic_samples = len(training_smic_list)

training_smic_labels = np.zeros((training_smic_samples, ), dtype = int)

training_smic_labels[0:66] = 0
training_smic_labels[66:116] = 1
training_smic_labels[116:159] = 2

training_smic_labels = np_utils.to_categorical(training_smic_labels, 3)

In [ ]:
training_smic_data = [training_smic_list, training_smic_labels]
(training_frames_samm, traininglabels_samm) = (training_smic_data[0], training_smic_data[1])
training_smic_set = np.zeros((training_smic_samples, 1, image_rows, image_columns, frames_Count))

for h in range(training_smic_samples):
    training_smic_set[h][0][:][:][:] = training_frames_samm[h,:,:,:]

In [ ]:
training_smic_set = training_smic_set.astype('float32')
training_smic_set -= np.mean(training_smic_set)
training_smic_set /= np.max(training_smic_set)

In [ ]:
training_smic_set.shape

#### Save Numpy Arrays

In [ ]:
# Save Nump Arrays to save time
np.save('/Thesis/MicroExpSTCNN/Training_dataset/microexp_smic_images.npy', training_smic_set)
np.save('/Thesis/MicroExpSTCNN/Training_dataset/microexp_smic_labels.npy', training_smic_labels)

#### Loading Numpy arrays

In [ ]:
training_smic_set = None
training_smic_labels = None
training_smic_set  = np.load('/Thesis/MicroExpSTCNN/Training_dataset/microexp_smic_images.npy')
training_smic_labels = np.load('/Thesis/MicroExpSTCNN/Training_dataset/microexp_smic_labels.npy')

training_smic_set.shape

#### Model

In [ ]:
model = Sequential()
model.add(Convolution3D(32, (3, 3, 15), input_shape=(1, image_rows, image_columns, frames_Count), activation='relu'))
model.add(MaxPooling3D(pool_size=(3, 3, 3)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3))
model.add(Activation('softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'SGD', metrics = ['accuracy'])

model.summary()

In [ ]:
# Load pre-trained weights

#model.load_weights('/Thesis/MicroExpSTCNN/Training_dataset/weights_microexpstcnn/weights-improvement-53-0.88.hdf5')

#model.load_weights('/Thesis/MicroExpSTCNN/CASME_SQUARE/weights-improvement-53-0.88.hdf5')

#### Creating checkpoints

In [ ]:
filepath="/Thesis/MicroExpSTCNN/Training_dataset_smic/weights_microexpstcnn_smic/weights-improvement-{epoch:02d}-{val_accuracy:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

#### Spliting the dataset into training and validation sets

In [ ]:
train_images, validation_images, train_labels, validation_labels =  train_test_split(training_smic_set, training_smic_labels, test_size=0.2, random_state=4)

#### Save validation set in a numpy array

In [ ]:
np.save('/Thesis/MicroExpSTCNN/Training_dataset_smic/microexpstcnn_val_images.npy', validation_images)
np.save('/Thesis/MicroExpSTCNN/Training_dataset_smic/microexpstcnn_val_labels.npy', validation_labels)

#### Load validation set from numpy array

In [ ]:
validation_images = np.load('/Thesis/MicroExpSTCNN/Training_dataset_smic/microexpstcnn_val_images.npy')
validation_labels = np.load('/Thesis/MicroExpSTCNN/Training_dataset_smic/microexpstcnn_val_labels.npy')

#### Pre-Trained data

In [ ]:
#validation_images = np.load('/Thesis/MicroExpSTCNN/CASME_SQUARE/microexpstcnn_val_images.npy')
#validation_labels = np.load('/Thesis/MicroExpSTCNN/CASME_SQUARE/microexpstcnn_val_labels.npy')

#### Training the model

In [ ]:
hist = model.fit(train_images, train_labels, validation_data = (validation_images, validation_labels), callbacks=callbacks_list, batch_size = 16, epochs = 100, shuffle=True)


#### Finding Confusion Matrix

In [ ]:
predictions = model.predict(validation_images)
predictions_labels = np.argmax(predictions, axis=1)
validation_labels = np.argmax(validation_labels, axis=1)
cfm = confusion_matrix(validation_labels, predictions_labels)
print (cfm)